In [1]:
import os
import librosa
import numpy as np
import sagemaker_pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, FloatType

In [2]:
from pyspark import SparkContext, SparkConf
from sagemaker_pyspark import classpath_jars
from pyspark.sql.functions import create_map, struct
from pydub import AudioSegment

In [3]:
LANDED = '/home/rlfo/Documents/pessoal/bigdatamusic/landed/'
RAW = '/home/rlfo/Documents/pessoal/bigdatamusic/raw/'

In [4]:
classpath = ":".join(sagemaker_pyspark.classpath_jars())

builder = SparkSession.builder.appName("MUSIC SPARK")
builder.config(
    "spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.extraClassPath", classpath)
builder.config("spark.driver.memory", "1g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")


builder.master("local[*]")


In [5]:
def audio_to_wav(file):
    dst = file.replace('.mp3',"")+".wav"
    sound = AudioSegment.from_mp3(file)
    sound.export(dst, format="wav")
    return dst
    

In [6]:
def extract_important_feature_music(file):
    
    songname = file.split('/')[0::-1][0]
    y, sr = librosa.load(file, mono=True, duration=30)
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    spec_cent = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    
    chroma_stft = np.array2string(chroma_stft, precision=4, separator=',',suppress_small=True)
    spec_cent = np.array2string(spec_cent, precision=4, separator=',',suppress_small=True)
    spec_bw = np.array2string(spec_bw, precision=4, separator=',',suppress_small=True)
    rolloff = np.array2string(rolloff, precision=4, separator=',',suppress_small=True)
    zcr = np.array2string(zcr, precision=4, separator=',',suppress_small=True)
    
    
    to_append = f'{songname};{chroma_stft};{spec_cent};{spec_bw};{rolloff};{zcr}'    
    return to_append
        

In [7]:
spark = builder.getOrCreate()

In [8]:
all_music = [f'{LANDED}{file}' for file in os.listdir(LANDED) if '.mp3' in file]

In [9]:
all_music

['/home/rlfo/Documents/pessoal/bigdatamusic/landed/Aux Fox - Ellie Goulding - Flux (Aux Fox Remix).mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/Young Nero - Change (Prod. Trippy T).mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/Young Nero - Beyond (Prod. Scott Storch).mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/wūsh - late nights with you.mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/PRDSEOHNO - OHNO - Lil Mama (prod. Fallen Roses and B Dom).mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/nymano - jazz and rain.mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/HIGH ON MUSIC - Danrell x Småland - Hostage.mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/Flipp Dinero - Leave Me Alone (Prod. by Young Forever x Cast Beats).mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/Cardi B - Money.mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusic/landed/ZZ - ICY (feat. Thorii).mp3',
 '/home/rlfo/Documents/pessoal/bigdatamusi

In [10]:
pipe_rdd_csv = spark.sparkContext.parallelize(all_music).map(audio_to_wav).map(extract_important_feature_music)

In [11]:
pipe_rdd_csv

PythonRDD[1] at RDD at PythonRDD.scala:52

In [12]:
schema = StructType([StructField('file_name', StringType(), True),
                     StructField('chroma', StringType(), True),
                     StructField('spec_cent', StringType(), True),
                     StructField('spec_bw', StringType(), True),
                     StructField('rolloff', StringType(), True),
                     StructField('zcr', StringType(), True)])

In [13]:
pipe_rdd_csv = pipe_rdd_csv.map(lambda x : x.split(";"))
rdd = spark.createDataFrame(pipe_rdd_csv,schema)
rdd.show(2)

+---------+------+---------+---------+---------+------+
|file_name|chroma|spec_cent|  spec_bw|  rolloff|   zcr|
+---------+------+---------+---------+---------+------+
|         |0.3115|1243.9466|1475.7774|2472.4184|0.0507|
|         |0.3245|1914.2462| 2062.584|4129.1167|0.0695|
+---------+------+---------+---------+---------+------+
only showing top 2 rows



In [14]:
RAW

'/home/rlfo/Documents/pessoal/bigdatamusic/raw/'

In [18]:
rdd.write.csv('/home/rlfo/raw',sep=';',mode='overwrite')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/rlfo/.local/share/virtualenvs/bigdatamusic-K9Mh5qpQ/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-0eed648b293e>", line 1, in <module>
    rdd.write.csv('/home/rlfo/raw',sep=';',mode='overwrite')
  File "/home/rlfo/.local/share/virtualenvs/bigdatamusic-K9Mh5qpQ/lib/python3.6/site-packages/pyspark/sql/readwriter.py", line 885, in csv
    self._jwrite.csv(path)
  File "/home/rlfo/.local/share/virtualenvs/bigdatamusic-K9Mh5qpQ/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/rlfo/.local/share/virtualenvs/bigdatamusic-K9Mh5qpQ/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/home/rlfo/.local/share/virtualenvs/bigdatamusic-K9Mh5qpQ/lib/python3.6/site-packages/py4j/protocol.py", 

Py4JJavaError: An error occurred while calling o218.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:224)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sca